In [24]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 Question-Answering 챗봇입니다. 주어질 질문에 대한 답변을 제공해주세요.'),
        MessagesPlaceholder(variable_name='chat_hitory'),    
        ('human', '#Question:\n{question}')
    ]
)

In [27]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini'
)

In [28]:
output_parser = StrOutputParser()

In [29]:
chain = prompt | llm | output_parser

In [30]:
store = {}  # 딕셔너리

In [31]:
def get_session_history(session_ids):
    print(f'[대화 세션 ID]: {session_ids}')  # abc123

    if session_ids not in store:
       store[session_ids] = ChatMessageHistory()
    
    return store[session_ids]

In [32]:
chain_with_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key='question', 
    history_messages_key='chat_hitory'
)

In [33]:
chain_with_history.invoke(
    {'question': '안녕. 내이름은 홍길동이야.'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

[대화 세션 ID]: abc123


'안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [34]:
chain_with_history.invoke(
    {'question': '오늘은 크리마스 이브야'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

[대화 세션 ID]: abc123


'맞아요! 크리스마스 이브는 많은 사람들에게 특별한 날이죠. 친구나 가족과 함께 보내시나요? 아니면 특별한 계획이 있으신가요?'

In [36]:
chain_with_history.invoke(
    {'question': '내일은 무슨날이지?'}, 
    config={'configurable': {'session_id': 'abc123'}}
)

[대화 세션 ID]: abc123


'내일은 크리스마스입니다! 많은 사람들이 기념하는 특별한 날이죠. 크리스마스를 어떻게 보내실 계획이신가요?'

In [38]:
chain_with_history.invoke(
    {'question': '내일은 무슨날이지?'}, 
    config={'configurable': {'session_id': 'abc12345'}}
)

[대화 세션 ID]: abc12345


'내일은 2023년 10월 6일입니다. 특정한 기념일이나 행사에 대한 정보가 필요하시면 말씀해 주세요!'